In [ ]:
#hide
from nbdev import *
from hecss.core import *

# Setup

>At the moment the HECSS sampler supports only VASP and ASAP3/OpenKIM calculators. However, it is explicitly written in such a way that it should be quite easy to extend it to other calculators (e.g. QE, AbInit). The VASP and ASAP calculators are simply the only ones tested so far. The following instructions cover only VASP. Contributions extending support to other calculators are welcomed and will be included in the distribution.

## VASP calculator

After installing of the HECSS package you need to configure it. This involves configuring ASE (Atomistic Simulation Environment) to be able to generate VASP input files. Refer to [ASE documentation](https://wiki.fysik.dtu.dk/ase/ase/calculators/vasp.html) for details. Usually yu need to:
1. Put your pseudopotential files in a dedicated directory tree as described  in the docs. My setup contains a directory with three subdirs for LDA, GGA, and PBE pseudopotentials:
```bash
potpaw  
potpaw_GGA  
potpaw_PBE
```
2. Setting the `VASP_PP_PATH` environment variable to the location of this directory
3. Preparing the `run-calc` script to execute vasp in your setup. The script *must* wait for the calculation to finish before it returns. The example script is included in the source and for SLURM queu manager may look similar to the following code:

```bash
#!/bin/bash

# This script should run vasp in current directory 
# and wait for the run to finish.
# 
# A generic line using SLURM would look like this:
#
# sbatch [job_params] -W vasp_running_script
#
# The "-W" param makes the sbatch command wait for the job to finish.


JN=`pwd`
JN=`basename ${JN}`

# Partition of the cluster
PART=small

# Number of nodes
N=1

# Number of MPI tasks
ntask=64

# Name the job after directory if no label is passed as first argument
if [ "${1}." != "." ]; then
  JN=${1}
fi

sbatch -W -J ${JN} -p $PART -N $N -n $ntask run-vasp-script
```
You need to adapt it to your setup. The script works properly if you can go to the directory with the prepared VASP configuration and execute `run-calc` and have it run vasp in the directory and finish after the VASP job ends.